# Generate the training data (wall + climber + pos)
- Wall: 15 different walls
- Climber: 300 climbers divided into casual, skilled, elite
- Pos: current position (hands position and feet position)

In [7]:
import pandas as pd

# Load the files
walls = pd.read_csv("climbing_data.csv")
climbers = pd.read_csv("all_climbers.csv")

In [5]:
climbers = climbers.drop(columns=["Unnamed: 0"])

In [7]:
from ast import literal_eval
walls["points"] = walls["points"].apply(literal_eval)

In [8]:
import sys
sys.path.append("../src")

from dataset_function import sample_grap_points_by_radius
# Generate the Training Data
times = 20
data = {
    "route": [],
    "climber": [],
    "hands": [],
    "feet": []
}

for w in walls["points"]:
    for _, c in climbers.iterrows():
        for n in range(times):
            hands, feet, center = sample_grap_points_by_radius(route=w,
                                                               climber=c,
                                                               max_trials=100)
            data["route"].append(w)
            data["climber"].append({"group": c["group"],
                                    "height": c["height"],
                                    "weight": c["weight"],
                                    "ape_index": c["ape_index"],
                                    "strength": c["strength"],
                                    "flexibility": c["flexibility"],
                                    "leg_len_factor": c["leg_len_factor"]})
            data["hands"].append(hands)
            data["feet"].append(feet)

In [10]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv("reachability_dataset.csv", index=False)

# Generate the training data (wall + climber + wall features + pos) (for Reachability Model with Holds features)

In [ ]:
# TODO: Create the correct training data: wall + climber + hands + feet + holds_features
"""
wall: 一面墙的全部抓点（中心坐标）
climber: 攀岩者的属性
hands: 目前手的抓点坐标
feet: 目前脚的抓点坐标
holds_features: 该墙每一个抓点的属性 e.g. [{'center_x': 1, 'center_y': 2, 'shape_area':...},{},{}]
"""

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from sklearn.neighbors import NearestNeighbors
import torch
from torch_geometric.data import Data
import sys
sys.path.append("../src")
from ast import literal_eval

In [2]:
walls_with_features = pd.read_csv("climbing_with_shape_features.csv")
climbers = pd.read_csv("all_climbers.csv").drop(columns=["Unnamed: 0"])
walls = pd.read_csv("climbing_data.csv").drop(columns=["Unnamed: 0"])
walls["points"] = walls["points"].apply(literal_eval)

In [82]:
shape_cols = [
    "center_x", "center_y", 
    "shape_area", "shape_perimeter", 
    "shape_aspect_ratio", "shape_circularity"
]

grouped = walls_with_features.groupby("filename")

combined_rows = []
for name, group in grouped:
    group = group.reset_index(drop=True)
    route = list(zip(group["center_x"], group["center_y"]))

    holds = [
        {col: row[col] for col in shape_cols}
        for i, row in group.iterrows()
    ]

    combined_rows.append({
        "name": name,
        "route": route,
        "holds_features": holds
    })

combined_df = pd.DataFrame(combined_rows)

In [83]:
import sys
sys.path.append("../src")

from dataset_function import sample_grap_points_by_radius
# Generate the Training Data
times = 20
data = {
    "route": [],
    "climber": [],
    "hands": [],
    "feet": [],
    "holds_features": []
}

for _, w in combined_df.iterrows():
    for _, c in climbers.iterrows():
        for n in range(times):
            hands, feet, center = sample_grap_points_by_radius(route=w["route"],
                                                               climber=c,
                                                               max_trials=100)
            data["route"].append(w["route"])
            data["holds_features"].append(w["holds_features"])
            data["climber"].append({"group": c["group"],
                                    "height": c["height"],
                                    "weight": c["weight"],
                                    "ape_index": c["ape_index"],
                                    "strength": c["strength"],
                                    "flexibility": c["flexibility"],
                                    "leg_len_factor": c["leg_len_factor"]})
            data["hands"].append(hands)
            data["feet"].append(feet)

In [84]:
df = pd.DataFrame(data)

In [86]:
df.to_csv("reachability_features_dataset.csv", index=False)

# Convert the Reachability features dataset to Graph for model training

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from sklearn.neighbors import NearestNeighbors
import torch
from torch_geometric.data import Data
import sys
sys.path.append("../src")
from ast import literal_eval
from reachability_model_holds_features_function import build_graph_reachability_features
from dataset_function import get_reachability_features_label

In [2]:
dataset = pd.read_csv("reachability_features_dataset.csv")

In [3]:
for name in dataset.columns:
    dataset[name] = dataset[name].apply(literal_eval)

In [4]:
graph_list = []

for _, row in dataset.iterrows():
    labels = get_reachability_features_label(row["route"],
                                             row["climber"],
                                             row["hands"],
                                             row["feet"],
                                             row["holds_features"]
                                             )
    if labels.count(0) / len(labels) < 0.95:
        data = build_graph_reachability_features(row["route"],
                                                 row["hands"],
                                                 row["feet"],
                                                 row["climber"],
                                                 row["holds_features"],
                                                 labels
                                                 )
        graph_list.append(data)

In [5]:
len(graph_list)

31359

In [6]:
torch.save(graph_list, "graph_list_holds_features_edgesF.pt")